# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources in the README.md file
- Happy learning!

In [154]:
# Import your libraries:

%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sn
import re



In this lab, we will explore a dataset that describes websites with different features and labels them either benign or malicious . We will use supervised learning algorithms to figure out what feature patterns malicious websites are likely to have and use our model to predict malicious websites.

# Challenge 1 - Explore The Dataset

Let's start by exploring the dataset. First load the data file:

In [155]:
websites = pd.read_csv('../website.csv')

#### Explore the data from an bird's-eye view.

You should already been very familiar with the procedures now so we won't provide the instructions step by step. Reflect on what you did in the previous labs and explore the dataset.

Things you'll be looking for:

* What the dataset looks like?
* What are the data types?
* Which columns contain the features of the websites?
* Which column contains the feature we will predict? What is the code standing for benign vs malicious websites?
* Do we need to transform any of the columns from categorical to ordinal values? If so what are these columns?

Feel free to add additional cells for your explorations. Make sure to comment what you find out.

In [156]:
websites.head()

## type column is the one which indicates if the website is malicious or not (1/0)

,URL,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CHARSET,SERVER,CONTENT_LENGTH,WHOIS_COUNTRY,WHOIS_STATEPRO,WHOIS_REGDATE,WHOIS_UPDATED_DATE,...,DIST_REMOTE_TCP_PORT,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,SOURCE_APP_BYTES,REMOTE_APP_BYTES,APP_PACKETS,DNS_QUERY_TIMES,Type
0,M0_109,16,7,iso-8859-1,nginx,263.0,None,None,10/10/2015 18:21,None,...,0,2,700,9,10,1153,832,9,2.0,1
1,B0_2314,16,6,UTF-8,Apache/2.4.10,15087.0,None,None,None,None,...,7,4,1230,17,19,1265,1230,17,0.0,0
2,B0_911,16,6,us-ascii,Microsoft-HTTPAPI/2.0,324.0,None,None,None,None,...,0,0,0,0,0,0,0,0,0.0,0
3,B0_113,17,6,ISO-8859-1,nginx,162.0,US,AK,7/10/1997 4:00,12/09/2013 0:45,...,22,3,3812,39,37,18784,4380,39,8.0,0
4,B0_403,17,6,UTF-8,None,124140.0,US,TX,12/05/1996 0:00,11/04/2017 0:00,...,2,5,4278,61,62,129889,4586,61,4.0,0


In [157]:
## the features are contained in some of the columns
## the rest of them are eliminated

print(websites.columns)

websites.drop(['DIST_REMOTE_TCP_PORT','SOURCE_APP_BYTES',
              'REMOTE_APP_BYTES'],axis=1,inplace=True)




Index(['URL', 'URL_LENGTH', 'NUMBER_SPECIAL_CHARACTERS', 'CHARSET', 'SERVER',
       'CONTENT_LENGTH', 'WHOIS_COUNTRY', 'WHOIS_STATEPRO', 'WHOIS_REGDATE',
       'WHOIS_UPDATED_DATE', 'TCP_CONVERSATION_EXCHANGE',
       'DIST_REMOTE_TCP_PORT', 'REMOTE_IPS', 'APP_BYTES', 'SOURCE_APP_PACKETS',
       'REMOTE_APP_PACKETS', 'SOURCE_APP_BYTES', 'REMOTE_APP_BYTES',
       'APP_PACKETS', 'DNS_QUERY_TIMES', 'Type'],
      dtype='object')


In [158]:
## find features with null values

null_values = websites.isnull().sum()
print(null_values)
content_length_nulls = null_values['CONTENT_LENGTH']
print(f'\nCONTENT_LENGTH feature has a percentage of null values of {(content_length_nulls/websites.shape[0])*100}')

URL                            0
URL_LENGTH                     0
NUMBER_SPECIAL_CHARACTERS      0
CHARSET                        0
SERVER                         1
CONTENT_LENGTH               812
WHOIS_COUNTRY                  0
WHOIS_STATEPRO                 0
WHOIS_REGDATE                  0
WHOIS_UPDATED_DATE             0
TCP_CONVERSATION_EXCHANGE      0
REMOTE_IPS                     0
APP_BYTES                      0
SOURCE_APP_PACKETS             0
REMOTE_APP_PACKETS             0
APP_PACKETS                    0
DNS_QUERY_TIMES                1
Type                           0
dtype: int64

CONTENT_LENGTH feature has a percentage of null values of 45.59236384053902


In [159]:
websites.dtypes

## all features have adequate types

URL                           object
URL_LENGTH                     int64
NUMBER_SPECIAL_CHARACTERS      int64
CHARSET                       object
SERVER                        object
CONTENT_LENGTH               float64
WHOIS_COUNTRY                 object
WHOIS_STATEPRO                object
WHOIS_REGDATE                 object
WHOIS_UPDATED_DATE            object
TCP_CONVERSATION_EXCHANGE      int64
REMOTE_IPS                     int64
APP_BYTES                      int64
SOURCE_APP_PACKETS             int64
REMOTE_APP_PACKETS             int64
APP_PACKETS                    int64
DNS_QUERY_TIMES              float64
Type                           int64
dtype: object

#### Next, evaluate if the columns in this dataset are strongly correlated.

In the Mushroom supervised learning lab we did recently, we mentioned we are concerned if our dataset has strongly correlated columns because if it is the case we need to choose certain ML algorithms instead of others. We need to evaluate this for our dataset now.

Luckily, most of the columns in this dataset are ordinal which makes things a lot easier for us. In the next cells below, evaluate the level of collinearity of the data.

We provide some general directions for you to consult in order to complete this step:

1. You will create a correlation matrix using the numeric columns in the dataset.

1. Create a heatmap using `seaborn` to visualize which columns have high collinearity.

1. Comment on which columns you might need to remove due to high collinearity.

In [160]:
corrMatrix = websites.corr()
corrMatrix.style.background_gradient(cmap='coolwarm')


,URL_LENGTH,NUMBER_SPECIAL_CHARACTERS,CONTENT_LENGTH,TCP_CONVERSATION_EXCHANGE,REMOTE_IPS,APP_BYTES,SOURCE_APP_PACKETS,REMOTE_APP_PACKETS,APP_PACKETS,DNS_QUERY_TIMES,Type
URL_LENGTH,1,0.917986,0.129745,-0.0384072,-0.0463669,-0.0264463,-0.0422637,-0.0337787,-0.0422637,-0.0685824,0.162104
NUMBER_SPECIAL_CHARACTERS,0.917986,1,0.214225,-0.0374591,-0.0471033,-0.0239136,-0.0400957,-0.0305973,-0.0400957,-0.0500478,0.280897
CONTENT_LENGTH,0.129745,0.214225,1,0.0784512,0.00477433,0.0512023,0.0741416,0.0910769,0.0741416,-0.0456438,-0.0908517
TCP_CONVERSATION_EXCHANGE,-0.0384072,-0.0374591,0.0784512,1,0.33108,0.45732,0.997796,0.990848,0.997796,0.349832,-0.0402018
REMOTE_IPS,-0.0463669,-0.0471033,0.00477433,0.33108,1,0.0231258,0.361104,0.304683,0.361104,0.548189,-0.0787831
APP_BYTES,-0.0264463,-0.0239136,0.0512023,0.45732,0.0231258,1,0.445822,0.468999,0.445822,0.0122213,-0.0112625
SOURCE_APP_PACKETS,-0.0422637,-0.0400957,0.0741416,0.997796,0.361104,0.445822,1,0.989285,1,0.410843,-0.034414
REMOTE_APP_PACKETS,-0.0337787,-0.0305973,0.0910769,0.990848,0.304683,0.468999,0.989285,1,0.989285,0.355716,-0.0328966
APP_PACKETS,-0.0422637,-0.0400957,0.0741416,0.997796,0.361104,0.445822,1,0.989285,1,0.410843,-0.034414
DNS_QUERY_TIMES,-0.0685824,-0.0500478,-0.0456438,0.349832,0.548189,0.0122213,0.410843,0.355716,0.410843,1,0.0687533


In [161]:
## there is a strong correlation between TCP_CONVERSATION_EXCHANGE and SOURCE_APP_PACKETS
## if we wanted to delete a second column, this would be SOURCE_APP_PACKETS
## the third column we could delete due to high collinearity is NUMBER_SPECIAL_CHARACTERS

# Challenge 2 - Remove Column Collinearity.

From the heatmap you created, you should have seen at least 3 columns that can be removed due to high collinearity. Remove these columns from the dataset.

Note that you should remove as few columns as you can. You don't have to remove all the columns at once. But instead, try removing one column, then produce the heatmap again to determine if additional columns should be removed. As long as the dataset no longer contains columns that are correlated for over 90%, you can stop. Also, keep in mind when two columns have high collinearity, you only need to remove one of them but not both.

In the cells below, remove as few columns as you can to eliminate the high collinearity in the dataset. Make sure to comment on your way so that the instructional team can learn about your thinking process which allows them to give feedback. At the end, print the heatmap again.

In [162]:
websites.drop(['TCP_CONVERSATION_EXCHANGE','SOURCE_APP_PACKETS','NUMBER_SPECIAL_CHARACTERS'],axis=1,inplace=True)
corrMatrix = websites.corr()
corrMatrix.style.background_gradient(cmap='coolwarm')

,URL_LENGTH,CONTENT_LENGTH,REMOTE_IPS,APP_BYTES,REMOTE_APP_PACKETS,APP_PACKETS,DNS_QUERY_TIMES,Type
URL_LENGTH,1,0.129745,-0.0463669,-0.0264463,-0.0337787,-0.0422637,-0.0685824,0.162104
CONTENT_LENGTH,0.129745,1,0.00477433,0.0512023,0.0910769,0.0741416,-0.0456438,-0.0908517
REMOTE_IPS,-0.0463669,0.00477433,1,0.0231258,0.304683,0.361104,0.548189,-0.0787831
APP_BYTES,-0.0264463,0.0512023,0.0231258,1,0.468999,0.445822,0.0122213,-0.0112625
REMOTE_APP_PACKETS,-0.0337787,0.0910769,0.304683,0.468999,1,0.989285,0.355716,-0.0328966
APP_PACKETS,-0.0422637,0.0741416,0.361104,0.445822,0.989285,1,0.410843,-0.034414
DNS_QUERY_TIMES,-0.0685824,-0.0456438,0.548189,0.0122213,0.355716,0.410843,1,0.0687533
Type,0.162104,-0.0908517,-0.0787831,-0.0112625,-0.0328966,-0.034414,0.0687533,1


In [163]:
websites.drop(['APP_PACKETS'],axis=1,inplace=True)
corrMatrix = websites.corr()
corrMatrix.style.background_gradient(cmap='coolwarm')

,URL_LENGTH,CONTENT_LENGTH,REMOTE_IPS,APP_BYTES,REMOTE_APP_PACKETS,DNS_QUERY_TIMES,Type
URL_LENGTH,1,0.129745,-0.0463669,-0.0264463,-0.0337787,-0.0685824,0.162104
CONTENT_LENGTH,0.129745,1,0.00477433,0.0512023,0.0910769,-0.0456438,-0.0908517
REMOTE_IPS,-0.0463669,0.00477433,1,0.0231258,0.304683,0.548189,-0.0787831
APP_BYTES,-0.0264463,0.0512023,0.0231258,1,0.468999,0.0122213,-0.0112625
REMOTE_APP_PACKETS,-0.0337787,0.0910769,0.304683,0.468999,1,0.355716,-0.0328966
DNS_QUERY_TIMES,-0.0685824,-0.0456438,0.548189,0.0122213,0.355716,1,0.0687533
Type,0.162104,-0.0908517,-0.0787831,-0.0112625,-0.0328966,0.0687533,1


# Challenge 3 - Handle Missing Values

The next step would be handling missing values. **We start by examining the number of missing values in each column, which you will do in the next cell.**

In [164]:
websites.isnull().sum()

URL                     0
URL_LENGTH              0
CHARSET                 0
SERVER                  1
CONTENT_LENGTH        812
WHOIS_COUNTRY           0
WHOIS_STATEPRO          0
WHOIS_REGDATE           0
WHOIS_UPDATED_DATE      0
REMOTE_IPS              0
APP_BYTES               0
REMOTE_APP_PACKETS      0
DNS_QUERY_TIMES         1
Type                    0
dtype: int64

If you remember in the previous labs, we drop a column if the column contains a high proportion of missing values. After dropping those problematic columns, we drop the rows with missing values.

#### In the cells below, handle the missing values from the dataset. Remember to comment the rationale of your decisions.

In [165]:
null_values = websites.isnull().sum()
print(null_values)
content_length_nulls = null_values['CONTENT_LENGTH']
print(f'\nCONTENT_LENGTH feature has a percentage of null values of {(content_length_nulls/websites.shape[0])*100}')

print(len(websites))
websites = websites.dropna()
print(len(websites))

URL                     0
URL_LENGTH              0
CHARSET                 0
SERVER                  1
CONTENT_LENGTH        812
WHOIS_COUNTRY           0
WHOIS_STATEPRO          0
WHOIS_REGDATE           0
WHOIS_UPDATED_DATE      0
REMOTE_IPS              0
APP_BYTES               0
REMOTE_APP_PACKETS      0
DNS_QUERY_TIMES         1
Type                    0
dtype: int64

CONTENT_LENGTH feature has a percentage of null values of 45.59236384053902
1781
967


In [166]:
print('We delete all rows with null values')

We delete all rows with null values


#### Again, examine the number of missing values in each column. 

If all cleaned, proceed. Otherwise, go back and do more cleaning.

In [167]:
websites.isnull().sum()

URL                   0
URL_LENGTH            0
CHARSET               0
SERVER                0
CONTENT_LENGTH        0
WHOIS_COUNTRY         0
WHOIS_STATEPRO        0
WHOIS_REGDATE         0
WHOIS_UPDATED_DATE    0
REMOTE_IPS            0
APP_BYTES             0
REMOTE_APP_PACKETS    0
DNS_QUERY_TIMES       0
Type                  0
dtype: int64

# Challenge 4 - Handle `WHOIS_*` Categorical Data

There are several categorical columns we need to handle. These columns are:

* `URL`
* `CHARSET`
* `SERVER`
* `WHOIS_COUNTRY`
* `WHOIS_STATEPRO`
* `WHOIS_REGDATE`
* `WHOIS_UPDATED_DATE`

How to handle string columns is always case by case. Let's start by working on `WHOIS_COUNTRY`. Your steps are:

1. List out the unique values of `WHOIS_COUNTRY`.
1. Consolidate the country values with consistent country codes. For example, the following values refer to the same country and should use consistent country code:
    * `CY` and `Cyprus`
    * `US` and `us`
    * `SE` and `se`
    * `GB`, `United Kingdom`, and `[u'GB'; u'UK']`

#### In the cells below, fix the country values as intructed above.

In [168]:
def changeCountry(name):
        if name=='Cyprus':
            return 'CY'
        elif name=='us':
            return 'US'
        elif name=='se':
            return 'SE'
        elif name=='United Kingdom':
            return 'GB'
        elif name=="[u'GB'; u'UK']":
            return 'GB'
        else:
            return name
    
websites['WHOIS_COUNTRY'] = websites['WHOIS_COUNTRY'].apply(changeCountry)
websites['WHOIS_COUNTRY'].unique()

array(['None', 'US', 'GB', 'UK', 'RU', 'AU', 'CA', 'PA', 'SE', 'IN', 'UG',
       'JP', 'SI', 'IL', 'AT', 'CN', 'BE', 'NO', 'TR', 'KY', 'BR', 'SC',
       'NL', 'FR', 'CZ', 'KR', 'UA', 'CH', 'HK', 'DE', 'IT', 'BS', 'CY',
       'BY', 'AE', 'IE', 'PH', 'UY'], dtype=object)

Since we have fixed the country values, can we convert this column to ordinal now?

Not yet. If you reflect on the previous labs how we handle categorical columns, you probably remember we ended up dropping a lot of those columns because there are too many unique values. Too many unique values in a column is not desirable in machine learning because it makes prediction inaccurate. But there are workarounds under certain conditions. One of the fixable conditions is:

#### If a limited number of values account for the majority of data, we can retain these top values and re-label all other rare values.

The `WHOIS_COUNTRY` column happens to be this case. You can verify it by print a bar chart of the `value_counts` in the next cell to verify:

In [173]:
top_countries = websites['WHOIS_COUNTRY'].value_counts()
top_countries

US      591
None    208
CA       45
GB       21
AU       16
PA       10
UK        6
IN        6
JP        6
CH        6
AT        4
KR        3
FR        3
CN        3
TR        3
BS        2
SC        2
DE        2
NO        2
CY        2
HK        2
BR        2
UA        2
IL        2
BE        2
SE        2
UY        2
NL        2
IE        1
PH        1
CZ        1
AE        1
SI        1
UG        1
IT        1
RU        1
KY        1
BY        1
Name: WHOIS_COUNTRY, dtype: int64

#### After verifying, now let's keep the top 10 values of the column and re-label other columns with `OTHER`.

In [196]:
def keepCountry(country,top_countries):
    if country in top_countries[:11] and country!='None':
        return country
    else:
        return 'OTHER'
    
websites['WHOIS_COUNTRY'] = websites['WHOIS_COUNTRY'].apply(keepCountry,top_countries=top_countries[:10])
websites['WHOIS_COUNTRY'].value_counts()

US       591
OTHER    260
CA        45
GB        21
AU        16
PA        10
IN         6
UK         6
JP         6
CH         6
Name: WHOIS_COUNTRY, dtype: int64

Now since `WHOIS_COUNTRY` has been re-labelled, we don't need `WHOIS_STATEPRO` any more because the values of the states or provinces may not be relevant any more. We'll drop this column.

In addition, we will also drop `WHOIS_REGDATE` and `WHOIS_UPDATED_DATE`. These are the registration and update dates of the website domains. Not of our concerns.

#### In the next cell, drop `['WHOIS_STATEPRO', 'WHOIS_REGDATE', 'WHOIS_UPDATED_DATE']`.

In [198]:
websites.drop(['WHOIS_STATEPRO', 'WHOIS_REGDATE', 'WHOIS_UPDATED_DATE'],axis=1,inplace=True)

# Challenge 5 - Handle Remaining Categorical Data & Convert to Ordinal

Now print the `dtypes` of the data again. Besides `WHOIS_COUNTRY` which we already fixed, there should be 3 categorical columns left: `URL`, `CHARSET`, and `SERVER`.

In [199]:
websites.dtypes

URL                    object
URL_LENGTH              int64
CHARSET                object
SERVER                 object
CONTENT_LENGTH        float64
WHOIS_COUNTRY          object
REMOTE_IPS              int64
APP_BYTES               int64
REMOTE_APP_PACKETS      int64
DNS_QUERY_TIMES       float64
Type                    int64
dtype: object

#### `URL` is easy. We'll simply drop it because it has too many unique values that there's no way for us to consolidate.

In [200]:
websites.drop(['URL'],axis=1,inplace=True)

#### Print the unique value counts of `CHARSET`. You see there are only a few unique values. So we can keep it as it is.

In [201]:
websites['CHARSET'].unique()

array(['iso-8859-1', 'UTF-8', 'us-ascii', 'ISO-8859-1', 'utf-8', 'None',
       'windows-1251', 'ISO-8859'], dtype=object)

`SERVER` is a little more complicated. Print its unique values and think about how you can consolidate those values.

#### Before you think of your own solution, don't read the instructions that come next.

In [203]:
websites['SERVER'].unique()

array(['nginx', 'Apache/2.4.10', 'Microsoft-HTTPAPI/2.0', 'None',
       'Apache/2', 'Apache/2.2.15 (Red Hat)',
       'Apache/2.4.23 (Unix) OpenSSL/1.0.1e-fips mod_bwlimited/1.4',
       'Apache/2.2.22', 'Apache/2.4.7 (Ubuntu)', 'Apache',
       'Apache/2.4.12 (Unix) OpenSSL/1.0.1e-fips mod_bwlimited/1.4',
       'Oracle-iPlanet-Web-Server/7.0', 'nginx/1.6.2', 'Heptu web server',
       'Microsoft-IIS/7.5', 'nginx/1.12.0',
       'Apache/2.4.25 (cPanel) OpenSSL/1.0.1e-fips mod_bwlimited/1.4',
       'tsa_c', 'Apache/2.2.22 (Debian)', 'Apache/2.4.25',
       'Apache/2.4.25 (Amazon) PHP/7.0.14', 'GSE',
       'Apache/2.4.23 (Unix) OpenSSL/0.9.8e-fips-rhel5 mod_bwlimited/1.4',
       'Apache/2.4.25 (Amazon) OpenSSL/1.0.1k-fips',
       'Apache/2.2.22 (Ubuntu)',
       'Apache/2.4.18 (Unix) OpenSSL/0.9.8e-fips-rhel5 mod_bwlimited/1.4',
       'Apache/2.4.6 (CentOS) PHP/5.6.8', 'AmazonS3', 'ATS',
       'Apache/2.2.27 (CentOS)', 'CherryPy/3.6.0',
       'Apache/2.2.15 (CentOS)', 'KHL',
   

![Think Hard](../think-hard.jpg)

In [ ]:
## i would only keep the server name, not the model

Although there are so many unique values in the `SERVER` column, there are actually only 3 main server types: `Microsoft`, `Apache`, and `nginx`. Just check if each `SERVER` value contains any of those server types and re-label them. For `SERVER` values that don't contain any of those substrings, label with `Other`.

At the end, your `SERVER` column should only contain 4 unique values: `Microsoft`, `Apache`, `nginx`, and `Other`.

In [204]:
def serverType(server):
    if 'Microsoft' in server:
        return 'Microsoft'
    elif 'Apache' in server:
        return 'Apache'
    elif 'nginx' in server:
        return 'nginx'
    else:
        return 'Other'
    
websites['SERVER'] = websites['SERVER'].apply(serverType)

In [213]:
websites['SERVER'].value_counts()

Apache       401
Other        221
Microsoft    180
nginx        165
Name: SERVER, dtype: int64

OK, all our categorical data are fixed now. **Let's convert them to ordinal data using Pandas' `get_dummies` function ([documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)).** Make sure you drop the categorical columns by passing `drop_first=True` to `get_dummies` as we don't need them any more. **Also, assign the data with dummy values to a new variable `website_dummy`.**

In [227]:
website_dummy = pd.get_dummies(websites,columns = ['SERVER','CHARSET','WHOIS_COUNTRY'],drop_first=True)

Now, inspect `website_dummy` to make sure the data and types are intended - there shouldn't be any categorical columns at this point.

In [228]:
website_dummy.dtypes

URL_LENGTH                int64
CONTENT_LENGTH          float64
REMOTE_IPS                int64
APP_BYTES                 int64
REMOTE_APP_PACKETS        int64
DNS_QUERY_TIMES         float64
Type                      int64
SERVER_Microsoft          uint8
SERVER_Other              uint8
SERVER_nginx              uint8
CHARSET_ISO-8859-1        uint8
CHARSET_None              uint8
CHARSET_UTF-8             uint8
CHARSET_iso-8859-1        uint8
CHARSET_us-ascii          uint8
CHARSET_utf-8             uint8
CHARSET_windows-1251      uint8
WHOIS_COUNTRY_CA          uint8
WHOIS_COUNTRY_CH          uint8
WHOIS_COUNTRY_GB          uint8
WHOIS_COUNTRY_IN          uint8
WHOIS_COUNTRY_JP          uint8
WHOIS_COUNTRY_OTHER       uint8
WHOIS_COUNTRY_PA          uint8
WHOIS_COUNTRY_UK          uint8
WHOIS_COUNTRY_US          uint8
dtype: object

# Challenge 6 - Modeling, Prediction, and Evaluation

We'll start off this section by splitting the data to train and test. **Name your 4 variables `X_train`, `X_test`, `y_train`, and `y_test`. Select 80% of the data for training and 20% for testing.**

In [238]:
from sklearn.model_selection import train_test_split

X = website_dummy.drop(['Type'],axis=1)
y = website_dummy['Type']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

#### In this lab, we will try two different models and compare our results.

The first model we will use in this lab is logistic regression. We have previously learned about logistic regression as a classification algorithm. In the cell below, load `LogisticRegression` from scikit-learn and initialize the model.

In [240]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression()

Next, fit the model to our training data. We have already separated our data into 4 parts. Use those in your model.

In [242]:
logistic_model.fit(X_train,y_train)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

finally, import `confusion_matrix` and `accuracy_score` from `sklearn.metrics` and fit our testing data. Assign the fitted data to `y_pred` and print the confusion matrix as well as the accuracy score

In [245]:
from sklearn.metrics import accuracy_score,confusion_matrix

y_pred = logistic_model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))


0.8865979381443299
[[163   0]
 [ 22   9]]


What are your thoughts on the performance of the model? Write your conclusions below.

In [246]:
# the model works quite well but it still failed on predicting the type of 22 websites


#### Our second algorithm is is K-Nearest Neighbors. 

Though is it not required, we will fit a model using the training data and then test the performance of the model using the testing data. Start by loading `KNeighborsClassifier` from scikit-learn and then initializing and fitting the model. We'll start off with a model where k=3.

In [254]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

To test your model, compute the predicted values for the testing sample and print the confusion matrix as well as the accuracy score.

In [255]:
y_pred = neigh.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))

0.8969072164948454
[[159   4]
 [ 16  15]]


#### We'll create another K-Nearest Neighbors model with k=5. 

Initialize and fit the model below and print the confusion matrix and the accuracy score.

In [256]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train,y_train)

y_pred = neigh.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))


0.9020618556701031
[[161   2]
 [ 17  14]]


Did you see an improvement in the confusion matrix when increasing k to 5? Did you see an improvement in the accuracy score? Write your conclusions below.

In [257]:
## yes, the accuracy improved and the confusion matrix gave better results:
## k=3 --> 20 mistakes
## k=5 --> 19 mistakes


# Bonus Challenge - Feature Scaling

Problem-solving in machine learning is iterative. You can improve your model prediction with various techniques (there is a sweetspot for the time you spend and the improvement you receive though). Now you've completed only one iteration of ML analysis. There are more iterations you can conduct to make improvements. In order to be able to do that, you will need deeper knowledge in statistics and master more data analysis techniques. In this bootcamp, we don't have time to achieve that advanced goal. But you will make constant efforts after the bootcamp to eventually get there.

However, now we do want you to learn one of the advanced techniques which is called *feature scaling*. The idea of feature scaling is to standardize/normalize the range of independent variables or features of the data. This can make the outliers more apparent so that you can remove them. This step needs to happen during Challenge 6 after you split the training and test data because you don't want to split the data again which makes it impossible to compare your results with and without feature scaling. For general concepts about feature scaling, click [here](https://en.wikipedia.org/wiki/Feature_scaling). To read deeper, click [here](https://medium.com/greyatom/why-how-and-when-to-scale-your-features-4b30ab09db5e).

In the next cell, attempt to improve your model prediction accuracy by means of feature scaling. A library you can utilize is `sklearn.preprocessing.RobustScaler` ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)). You'll use the `RobustScaler` to fit and transform your `X_train`, then transform `X_test`. You will use logistic regression to fit and predict your transformed data and obtain the accuracy score in the same way. Compare the accuracy score with your normalized data with the previous accuracy data. Is there an improvement?

In [258]:
from sklearn.preprocessing import RobustScaler

In [259]:
robust_model = RobustScaler()

In [266]:
train = robust_model.fit_transform(X_train)
test = robust_model.transform(X_test)